In [1]:
rm(list = ls())
setwd("/home/projects/14001280/PROJECTS/blood_microbiome")
require("Rsamtools")
require(Biostrings)
require(tidyverse)
require(foreach)
require(doParallel)
registerDoParallel(cores = 10)

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

L

In [2]:
bam_path <- "results/irep_analysis/raw_output/bam_files"
ref_path <- "data/irep_data/genome_references"
prefixes <- list.files(ref_path, ".fasta", full.names = F)
prefixes <- gsub(".fasta", "", prefixes)
prefixes

[1] "Achromobacter_xylosoxidans_NZ_CP043820.1"            
 [2] "Acinetobacter_baumannii_NZ_CP043953.1"               
 [3] "Bacillus_cereus_NZ_CP072774.1"                       
 [4] "Burkholderia_contaminans_GCF_004723625.1_ASM472362v1"
 [5] "Corynebacterium_segmentosum_NZ_LR134408.1"           
 [6] "Cutibacterium_acnes_NC_021085.1"                     
 [7] "Enterobacter_cloacae_NZ_CP009756.1"                  
 [8] "Fusobacterium_nucleatum_NZ_LN831027.1"               
 [9] "Fusobacterium_periodonticum_NZ_GG665898.1"           
[10] "Gardnerella_vaginalis_NZ_PKJK01000001.1"             
[11] "Haemophilus_parainfluenzae_NZ_GL872339.1"            
[12] "Human_mastadenovirus_C_NC_001405.1"                  
[13] "Lactobacillus_crispatus_NZ_CP039266.1"               
[14] "Microbacterium_PAMC_28756_NZ_CP014313.1"             
[15] "Microbacterium_PM5_NZ_CP022162.1"                    
[16] "Microbacterium_Y-01_NZ_CP024170.1"                   
[17] "Microbacterium_hominis_NZ_CP061344.1"                
[18] "Microbacterium_paraoxydans_NZ_LT629770.1"            
[19] "Neisseria_subflava_NZ_CP039887.1"                    
[20] "Paracoccus_yeei_NZ_CP024422.1"                       
[21] "Pseudomonas_mendocina_NZ_CP013124.1"                 
[22] "Ralstonia_solanacearum_NZ_CP088237.1"                
[23] "Rickettsia_felis_NZ_JSEL01000013.1"                  
[24] "Staphylococcus_cohnii_NZ_UHDA01000001.1"             
[25] "Staphylococcus_epidermidis_NZ_CP035288.1"            
[26] "Staphylococcus_haemolyticus_NZ_CP013911.1"           
[27] "Zhihengliuella_ISTPL4_NZ_CP025422.1"

In [3]:
morsels <- foreach (i = seq(length(prefixes))) %dopar% {
#     i <- 1
#     j <- 1
    prefix <- prefixes[i]
    bams <- list.files(str_glue("{bam_path}/{prefix}"))
    bams <- bams[!grepl(".bai", bams)]

    crumbs <- foreach (j = seq(length(bams))) %do% {
        bam <- bams[j]
        ref <- readDNAStringSet(str_glue("{ref_path}/{prefixes[i]}.fasta"))
        ref_length <- width(ref)
        params <- PileupParam(distinguish_strands = F, distinguish_nucleotides = F)
        pile_df <- pileup(str_glue("{bam_path}/{prefix}/{bam}"), pileupParam = params)
        
        n_covered1 <- pile_df %>%
            filter(count >= 1) %>%
            nrow()
        
        n_covered5 <- pile_df %>%
            filter(count >= 5) %>%
            nrow()

        perc_covered1 <- n_covered1 / ref_length * 100
        perc_covered5 <- n_covered5 / ref_length * 100
        
        return(tibble(prefix = prefix, 
                      perc_covered1 = perc_covered1,
                      perc_covered5 = perc_covered5))
    }
    
    return(bind_rows(crumbs))
}

cov_df <- bind_rows(morsels)

In [6]:
cov_df %>%
    arrange(desc(perc_covered1))

prefix,perc_covered1,perc_covered5
<chr>,<dbl>,<dbl>
Pseudomonas_mendocina_NZ_CP013124.1,69.178516,6.295949e+01
Achromobacter_xylosoxidans_NZ_CP043820.1,65.942462,1.854962e+01
Pseudomonas_mendocina_NZ_CP013124.1,65.814744,5.118415e+01
Achromobacter_xylosoxidans_NZ_CP043820.1,63.372644,1.331145e+01
Pseudomonas_mendocina_NZ_CP013124.1,61.543401,3.450724e+01
Burkholderia_contaminans_GCF_004723625.1_ASM472362v1,14.541461,4.117875e+00
Burkholderia_contaminans_GCF_004723625.1_ASM472362v1,11.237105,1.994434e+00
Achromobacter_xylosoxidans_NZ_CP043820.1,9.195231,1.075749e-01
Microbacterium_Y-01_NZ_CP024170.1,4.312364,4.389527e-02
